# Example: Optimal adversaries for dense MNIST model

This notebook gives an example where OMLT is used to find adversarial examples for a trained dense neural network. We follow the below steps:<br>
1.) A neural network with ReLU activation functions is trained to classify images from the MNIST dataset <br>
2.) OMLT is used to generate a mixed-integer encoding of the trained model using the big-M formulation <br>
3.) The model is optimized to find the maximum classification error (defined by an "adversarial" label) over a small input region <br>


## Library Setup
This notebook assumes you have a working PyTorch environment to train the neural network for classification. The neural network is then formulated in Pyomo using OMLT which therefore requires working Pyomo and OMLT installations.

The required Python libraries used this notebook are as follows: <br>
- `numpy`: used for manipulate input data <br>
- `torch`: the machine learning language we use to train our neural network
- `torchvision`: a package containing the MNIST dataset
- `pyomo`: the algebraic modeling language for Python, it is used to define the optimization model passed to the solver
- `omlt`: the package this notebook demonstates. OMLT can formulate machine learning models (such as neural networks) within Pyomo

In [2]:
#Import requisite packages
#data manipulation
import numpy as np
import tempfile

#pytorch for training neural network
import torch, torch.onnx
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

#pyomo for optimization
import pyomo.environ as pyo

#omlt for interfacing our neural network with pyomo
from omlt import OmltBlock
from omlt.neuralnet import FullSpaceNNFormulation
from omlt.io.onnx import write_onnx_model_with_bounds, load_onnx_neural_network_with_bounds

## Import the Data and Train a Neural Network

We begin by loading the MNIST dataset as `DataLoader` objects with pre-set training and testing batch sizes:


In [3]:
#set training and test batch sizes
train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}

#build DataLoaders for training and test sets
dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transforms.ToTensor())
dataset2 = datasets.MNIST('../data', train=False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

Next, we define the structure of the dense neural network model:

In [4]:
hidden_size = 50

class Net(nn.Module):
    #define layers of neural network
    def __init__(self):
        super().__init__()
        self.hidden1  = nn.Linear(784, hidden_size)
        self.hidden2  = nn.Linear(hidden_size, hidden_size)
        self.output  = nn.Linear(hidden_size, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    #define forward pass of neural network
    def forward(self, x):
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.softmax(x)      
        return x

We next define simple functions for training and testing the neural network:

In [5]:
#training function computes loss and its gradient on batch, and prints status after every 200 batches
def train(model, train_loader, optimizer, epoch):
    model.train(); criterion = nn.NLLLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.view(-1, 28*28))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

#testing function computes loss and prints overall model accuracy on test set
def test(model, test_loader):
    model.eval(); criterion = nn.NLLLoss( reduction='sum')
    test_loss = 0; correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.view(-1, 28*28))
            test_loss += criterion(output, target).item()  
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))            

Finally, we train the neural network on the dataset.
Training here is performed using the `Adadelta` optimizer for five epochs.

In [6]:
#define model and optimizer
model = Net()
optimizer = optim.Adadelta(model.parameters(), lr=1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

#train neural network for five epochs
for epoch in range(5):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)
    scheduler.step()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.300430
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.304656
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.186269
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.118478
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.309007

Test set: Average loss: 0.1444, Accuracy: 9553/10000 (96%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.041970
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.065082
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.059829
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.075850
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.148626

Test set: Average loss: 0.1338, Accuracy: 9559/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.072309
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.037450
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.097436
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.043346
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.053814

Test set: Average loss: 0.1035, Accuracy: 9680/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.140635
Train Epoch: 3 [12800/60000 (21%)]	Lo

## Build a MIP Formulation for the Trained Neural Network

We are now ready to use OMLT to formulate the trained model within a Pyomo optimization model. The nonsmooth ReLU activation function requires using a full-space representation, which uses the `NeuralNetworkFormulation` object.

First, we define a neural network without the final `LogSoftmax` activation. Although this activation helps greatly in training the neural network model, it is not trivial to encode in the optimization model. The ranking of the output labels remains the same without the activation, so it can be omitted when finding optimal adversaries. 

In [7]:
class NoSoftmaxNet(nn.Module):
    #define layers of neural network
    def __init__(self):
        super().__init__()
        self.hidden1  = nn.Linear(784, hidden_size)
        self.hidden2  = nn.Linear(hidden_size, hidden_size)
        self.output  = nn.Linear(hidden_size, 10)
        self.relu = nn.ReLU()

    #define forward pass of neural network
    def forward(self, x):
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.output(x)
        return x

#create neural network without LogSoftmax and load parameters from existing model
model2 = NoSoftmaxNet()
model2.load_state_dict(model.state_dict())

<All keys matched successfully>

Next, we define an instance of the optimal adversary problem. We formulate the optimization problem as: <br>

$
\begin{align*} 
& \max_x \ y_k - y_j \\
& s.t. y_k = N_k(x) \\ 
&\quad |x - \bar{x}|_\infty \leq 0.05
\end{align*}
$

where $\bar{x}$ corresponds to an image in the test dataset with true label `j`, and $N_k(x)$ is the value of the neural network output corresponding to adversarial label `k` given input `x`. PyTorch needs to trace the model execution to export it to ONNX, so we also define a dummy input tensor `x_temp`.

In [8]:
#load image and true label from test set with index 'problem_index'
problem_index = 0
image = dataset2[problem_index][0].view(-1,28*28).detach().numpy()
label = dataset2[problem_index][1]

#define input region defined by infinity norm
epsilon_infty = 5e-2
lb = np.maximum(0, image - epsilon_infty)
ub = np.minimum(1, image + epsilon_infty)

#save input bounds as dictionary
input_bounds = {}
for i in range(28*28):
    input_bounds[i] = (float(lb[0][i]), float(ub[0][i])) 
    
#define dummy input tensor    
x_temp = dataset2[problem_index][0].view(-1,28*28)

We can now export the PyTorch model as an ONNX model and use `load_onnx_neural_network_with_bounds` to load it into OMLT.

In [9]:
with tempfile.NamedTemporaryFile(suffix='.onnx', delete=False) as f:
    #export neural network to ONNX
    torch.onnx.export(
        model2,
        x_temp,
        f,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )
    #write ONNX model and its bounds using OMLT
    write_onnx_model_with_bounds(f.name, None, input_bounds)
    #load the network definition from the ONNX model
    network_definition = load_onnx_neural_network_with_bounds(f.name)

As a sanity check before creating the optimization model, we can print the properties of the neural network layers from `network_definition`. This allows us to check input/output sizes, as well as activation functions.

In [10]:
for layer_id, layer in enumerate(network_definition.layers):
    print(f"{layer_id}\t{layer}\t{layer.activation}")

0	InputLayer(input_size=[784], output_size=[784])	linear
1	DenseLayer(input_size=[784], output_size=[50])	relu
2	DenseLayer(input_size=[50], output_size=[50])	relu
3	DenseLayer(input_size=[50], output_size=[10])	linear


Finally, we can load `network_definition` as a full-space `FullSpaceNNFormulation` object.

In [11]:
formulation = FullSpaceNNFormulation(network_definition)

## Solve Optimal Adversary Problem in Pyomo

We now encode the trained neural network in a Pyomo model from the `FullSpaceNNFormulation` object. 

In [12]:
#create pyomo model
m = pyo.ConcreteModel()

#create an OMLT block for the neural network and build its formulation
m.nn = OmltBlock()
m.nn.build_formulation(formulation) 

Next, we define an adversarial label as the true label plus one (or zero if the true label is nine), as well as the objective function for optimization.

In [13]:
adversary = (label + 1) % 10
m.obj = pyo.Objective(expr=(-(m.nn.outputs[adversary]-m.nn.outputs[label])))

Finally, we solve the optimal adversary problem using a mixed-integer solver.

In [14]:
pyo.SolverFactory('gurobi').solve(m, tee=True)

Academic license - for non-commercial use only - expires 2023-01-12
Using license file /Users/calvintsay/gurobi.lic
Read LP format model from file /var/folders/pc/7mzx4b956_lb2l8_ryngwydc0000gn/T/tmpzltt46i4.pyomo.lp
Reading time = 0.05 seconds
x2693: 2109 rows, 2693 columns, 46307 nonzeros
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2109 rows, 2693 columns and 46307 nonzeros
Model fingerprint: 0x648bfaa5
Variable types: 2593 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [4e-07, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-03, 7e+01]
  RHS range        [9e-04, 8e+00]
Presolve removed 1812 rows and 1682 columns
Presolve time: 0.13s
Presolved: 297 rows, 1011 columns, 33818 nonzeros
Variable types: 941 continuous, 70 integer (70 binary)

Root relaxation: objective -8.339613e+00, 146 iterations, 0.01 seconds

    Nodes    |    Cur

{'Problem': [{'Name': 'x2693', 'Lower bound': 3.604549910060278, 'Upper bound': 3.6045499100602787, 'Number of objectives': 1, 'Number of constraints': 2109, 'Number of variables': 2693, 'Number of binary variables': 100, 'Number of integer variables': 100, 'Number of continuous variables': 2593, 'Number of nonzeros': 46307, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '1.632228136062622', 'Error rc': 0, 'Time': 1.9378628730773926}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}